In [1]:
import numpy as np
import numpy.linalg as nlin
import tensorflow as tf

img = tf.constant(np.random.randn(1, 180, 180, 1), dtype = tf.float32)
#conv1 = tf.layers.conv2d(img, filters = 64, kernel_size = [8,8], strides = [1,1], padding = 'SAME')
conv1 = tf.extract_image_patches(img, ksizes = [1,8,8,1], strides = [1,1,1,1], rates = [1,1,1,1], padding = 'VALID')

/Users/e.akhiyarova/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pH, pW = conv1.get_shape()[1:3]


In [3]:
def knn(input, Nbrs = 10):
    Ni, pH, pW, K = input.get_shape()
    X = tf.reshape(input, [Ni, pH*pW, K])
    final_indlist=[]
    batchlist = []
    for i in range(Ni):
        xklist = []
        indlist = []
        for j in range(pH*pW):
           # print(j)
            elem = X[i,j]
            distance = tf.sqrt(tf.reduce_sum(tf.square(X[i]-elem), reduction_indices = 1))
            values, indices = tf.nn.top_k(tf.negative(distance), k=Nbrs, sorted = False)
            xklist.append(tf.gather(X[i], indices = indices, axis = 0))
            indlist.append(indices)
        final_indlist.append(tf.stack(indlist))
        batchlist.append(tf.stack(xklist))
    return tf.stack(batchlist), tf.stack(final_indlist)

def iknn(input, idx,  pH, pW):
    Ni, pHpW, Nbrs, K = input.get_shape()
    idx_sorted, I = tf.nn.top_k(tf.reshape(idx, [Ni, pHpW*Nbrs]), k=pHpW*Nbrs)
    idx_sorted = tf.reverse(idx_sorted, axis = [1])
    I = tf.reverse(I, axis = [1])
    final_list = []
    for im_id in range(Ni):
        _,_, n = tf.unique_with_counts(idx_sorted[im_id])
        #for i in range(pHpW):
        tensor_list = []
        k=0
        for i in range(pHpW):
            indices = I[im_id, k:k+n[i]]
            d = tf.gather(tf.reshape(D[im_id], [1, pHpW*Nbrs, K]), indices = indices, axis = 1)
            tensor_list.append(tf.reduce_sum(d, axis=1)/tf.cast(n[i], tf.float32))
            k += n[i]
           
        final_list.append(tf.concat(tensor_list, axis = 0))

    iD = tf.stack(final_list, axis = 0)
    return tf.reshape(iD, [Ni, pH, pW, K])

def iknn_v2(input, idx, pH, pW):
    Ni, pHpW, Nbrs, K = input.get_shape()
    idx_sorted, I = tf.nn.top_k(tf.reshape(idx, [Ni, pHpW*Nbrs]), k=pHpW*Nbrs)
    idx_sorted = tf.reverse(idx_sorted, axis = [1])
    I = tf.reverse(I, axis = [1])
    print(idx_sorted)
    print(I)
    #nlist = []
    tensor_list = []
    k=tf.zeros(Ni, dtype = tf.int32)
    for i in range(pHpW):
        ni = tf.reduce_sum(tf.cast(tf.equal(tf.reshape(idx, [Ni, pHpW*Nbrs]), i), tf.int32), axis = 1 )
        indices = I[:, k:k+ni]
        d = tf.gather(tf.reshape(D, [Ni, pHpW*Nbrs, K]), indices = indices, axis = 1)
        tensor_list.append(tf.reduce_sum(d, axis=1)/tf.cast(ni, tf.float32))
        k = k+ni
    iD = tf.concat(tensor_list)
    

In [4]:
%%time
D, idx = knn(conv1, Nbrs=10)



CPU times: user 8min 20s, sys: 1min 4s, total: 9min 25s
Wall time: 9min 33s


In [9]:
Ni, pHpW, Nbrs, K = D.get_shape()

In [ ]:
%%time
idx_sorted = tf.contrib.framework.sort(tf.reshape(idx, [Ni, pHpW*Nbrs]), axis=-1, direction='ASCENDING', name=None)
I = tf.contrib.framework.argsort(tf.reshape(idx, [Ni, pHpW*Nbrs]), axis=-1, direction='ASCENDING', name=None)
with tf.Session() as sess:
    idx_s=sess.run(idx_sorted)
    I_s = sess.run(I)

In [ ]:
%%time
idx_sorted1, I1 = tf.nn.top_k(tf.reshape(idx, [Ni, pHpW*Nbrs]), k=pHpW*Nbrs)
with tf.Session() as sess:
    idx_1=sess.run(idx_sorted1)
    I_1 = sess.run(I1)

In [ ]:
%%time
iD = iknn(D, idx, pH.value, pW.value)

In [ ]:
with tf.Session() as sess:
    print(nlin.norm(sess.run(iD-conv1)))

In [ ]:
nlist = []
for i in range(pHpW):
    nlist.append(tf.reduce_sum(tf.cast(tf.equal(tf.reshape(idx, [Ni, pHpW*Nbrs]), i), tf.int32), axis = 1 ))

n = tf.stack(nlist, axis = 1)
res

In [ ]:
y,ind, n_f = tf.unique_with_counts(tf.reshape(idx[1], [pHpW*Nbrs]))


In [ ]:
with tf.Session() as sess:
    print(sess.run(res)[1])
    print(sess.run(y))
    print(sess.run(ind))
    print(sess.run(n_f))

In [ ]:
import tensorflow as tf

#X = tf.constant([6, 3, 3, 3, 0, 1, 3, 6, 7])
res = tf.reduce_sum(tf.cast(tf.equal(idx, 0), tf.int32))

In [ ]:
res

In [ ]:
conv1

In [ ]:
%%time
iD = iknn(D, idx, 9, 9)

In [ ]:
D.shape

In [ ]:
conv1

In [ ]:
iD